# Example for using Kernel CMSM  Model with sklearn-like API

In [29]:
import sys, os
sys.path.insert(0, os.pardir)

### Sample face dataset
 - Put Face_data.mat under sample_dataset

In [38]:
import h5py
import numpy as np

file_path = "../sample_dataset/Face_data.mat"

def load_data(file_name):
    """Load data from a .mat file

    Args:
        file_name (str): The name of the .mat file to load

    Returns:
        train_X (ndarray): The training data
        train_y (ndarray): The training labels
        test_X (ndarray): The test data
        test_y (ndarray): The test labels
    """
    # Load the .mat file
    with h5py.File(file_name, "r") as f:
        # Extract the data from the file
        train_X = np.array(f["X1"])
        train_y = np.arange(len(train_X))
        test_X = np.array(f["X2"])
        test_X = test_X.reshape(-1, test_X.shape[-2], test_X.shape[-1])
        test_y = np.array([[i] * 36 for i in range(10)]).flatten()

    return train_X, train_y, test_X, test_y

train_X, train_y, test_X, test_y = load_data(file_path)

print("Training data shape: {}".format(train_X.shape))
print("Training labels shape: {}".format(train_y.shape))
print("Test data shape: {}".format(test_X.shape))
print("Test labels shape: {}".format(test_y.shape))

Training data shape: (10, 100, 256)
Training labels shape: (10,)
Test data shape: (360, 25, 256)
Test labels shape: (360,)


### Training

In [31]:
# eps = 5e-1
# rs = np.random.RandomState(seed=100)
# train_X = [_X + eps * rs.randn(*_X[0].shape) for _X in train_X]
# test_X = [_X + eps * rs.randn(*_X[0].shape) for _X in test_X]

In [32]:
# Set the amount of random noise to add to the data
noise_scale = 5e-1

# Initialize a random number generator with a fixed seed
rng = np.random.RandomState(seed=100)

# Add random noise to the training data
train_X = [_X + noise_scale * rng.randn(*_X[0].shape) for _X in train_X]

# Add random noise to the test data
test_X = [_X + noise_scale * rng.randn(*_X[0].shape) for _X in test_X]

In [33]:
from cvt.models import KernelMSM
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

model = KernelMSM(n_subdims=5, sigma=100, faster_mode=True)
model.fit(train_X, train_y)

### Evaluation

In [34]:
pred = model.predict(test_X)
print(f"pred: {pred}\ntrue: {test_y}\naccuracy: {(pred == test_y).mean()}")

pred: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 7 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 4 1 1 4 1 1 1 1 1 1 1 1 1 7 1 1 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 6 2 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 3 3 4 4 4 4
 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5
 5 5 5 5 5 5 5 8 5 8 5 5 5 5 5 5 8 5 8 5 5 5 9 5 5 5 5 5 8 5 5 6 6 6 6 6 5
 6 6 6 6 6 6 6 9 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7
 7 7 7 7 7 4 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 0 7 7 7 7 7 8 8 8 8 8 8 8 8
 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 6 6 6 9 9 9 9 9 9
 9 9 6 9 9 9 9 6 9 9 9 9 6 6 6 9 6 6 6 9 9 9 9 9 9 9 9]
true: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3